In [1]:
import pandas as pd
import requests

In [2]:
#---------------------------------------------------
#
# Config
#
#---------------------------------------------------

# This file should be automatically generated from "1. Gaze Projections and AOIs Mapping"
GAZE_DATA_WITH_AOIS_FILE = "data/gazeDataWithAOIs.csv"

In [3]:
#----------------------------------------------------------------------------------------
#
#
# 1. Oculomotor event detection (i.e., Fixations and saccades)
# 
#
#----------------------------------------------------------------------------------------

In [4]:
# Read gaze data with AOIs using pandas library
data = pd.read_csv(GAZE_DATA_WITH_AOIS_FILE)
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

C:\Users\johan\AppData\Local\Temp\ipykernel_25144\3739489302.py:2: DtypeWarning: Columns (8,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(GAZE_DATA_WITH_AOIS_FILE)


In [5]:
# Preview data
data.head()

,Row,Respondent,Age,Gender,Group,Calibration,Timestamp,EventSource,SlideEvent,StimType,Duration,CollectionPhase,SourceStimuliName,EventSource.1,ET_GazeLeftx,ET_GazeLefty,ET_GazeRightx,ET_GazeRighty,ET_PupilLeft,ET_PupilRight,ET_TimeSignal,ET_DistanceLeft,ET_DistanceRight,ET_CameraLeftX,ET_CameraLeftY,ET_CameraRightX,ET_CameraRightY,ET_ValidityLeft,ET_ValidityRight,GazeX,GazeY,SWA_Stimulus_Journalist_List_1,SWA_Stimulus_Status_invited1,SWA_Stimulus_Cancel_add,SWA_Stimulus_Add_1,SWA_Stimulus_Status_Request2,SWA_Stimulus_Navigation_2,SWA_Stimulus_Status_Published3,SWA_Stimulus_EventInfo_1,SWA_Stimulus_Confirm_add,SWA_Stimulus_Status_Published2,SWA_Stimulus_Journalist_List_6,SWA_Stimulus_Status_Published_6,SWA_Stimulus_Status_Published_5,SWA_Stimulus_Status Request1,SWA_Stimulus_EventInfo_2,SWA_Stimulus_Status_Published1,SWA_Stimulus_Journalist_List5,SWA_Stimulus_Question_Task1,SWA_Stimulus_Journalist_List4,SWA_Stimulus_EventInfo_3,SWA_Stimulus_Status_Published_4,SWA_Stimulus_Navigation_1,SWA_Stimulus_Journalist_selection,SWA_Stimulus_Event_Details,SWA_Stimulus_Journalist_List_3,SWA_Stimulus_Available_Journalists,SWA_Stimulus_Add_2,SWA_Stimulus_Journalist_List_2,SWA_Stimulus2_Current_Selection,SWA_Stimulus2_View button,SWA_Stimulus2_Navigation_2,SWA_Stimulus2_EventInfo,SWA_Stimulus2_Edit_button,SWA_Stimulus2_Submission_information,SWA_Stimulus2_Question_Task2,SWA_Stimulus2_Question,SWA_Stimulus2_Navigation_3,SWA_Stimulus2_Help_button,SWA_Stimulus2_Navigation_1,SWA_Stimulus2_Submission,SWA_Stimulus2_Question_Editor,SWA_Stimulus2_Published_Report_1,SWA_Stimulus2_Question_Control
0,1,P1,0,OTHER,Default,Good,113.4082,1.0,StartSlide,TestImage,300000.0,StimuliDisplay,SWA_Stimulus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,P1,0,OTHER,Default,Good,116.3702,NaN,NaN,NaN,NaN,NaN,SWA_Stimulus,1.0,888.0,515.0,1010.0,425.0,NaN,NaN,124.985,619.543823,642.581055,0.619945,0.423148,0.458277,0.416878,0.0,0.0,949.0,470.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,P1,0,OTHER,Default,Good,124.6983,NaN,NaN,NaN,NaN,NaN,SWA_Stimulus,1.0,888.0,515.0,1012.0,420.0,4.466522,4.194916,133.313,619.592834,642.683228,0.619940,0.423088,0.458230,0.416665,0.0,0.0,950.0,467.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,P1,0,OTHER,Default,Good,133.0560,NaN,NaN,NaN,NaN,NaN,SWA_Stimulus,1.0,902.0,491.0,1012.0,420.0,NaN,NaN,141.671,619.556641,642.699036,0.619840,0.423102,0.458169,0.416592,0.0,0.0,957.0,455.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,P1,0,OTHER,Default,Good,141.4015,NaN,NaN,NaN,NaN,NaN,SWA_Stimulus,1.0,900.0,497.0,1012.0,443.0,NaN,NaN,150.015,619.610413,642.775269,0.619859,0.422902,0.458232,0.416609,0.0,0.0,956.0,470.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
#Keep only the gaze data coming from the eye-tracking device
data = data[ data['EventSource.1'] == 1 ]

In [7]:
#Fill n/a values with -1 (for JSON Compliance in requests) 
data = data.fillna(-1) 

In [10]:
# Oculomotor event detection (i.e., Fixations and saccades)
# Using customizable I-VT (Velocity Threshold) filter for fixation and saccade detection

dataWithEvents = None

# Iteratate over the participants and the apply I-VT filter on the data of each participant
for participant in data['Respondent'].unique():
    
    # filter in the participant data
    ParticipantData = data[data['Respondent']==participant]
    
    print(f'Applying fixation filter on the data for participant: {participant}')
    config = {
                    "fixationFilter" : "IVT",
                    "screenResolutionWidth" : 1920, 
                    "screenResolutionHeight" : 1080, 
                    "monitorSize" : 24, 
                    "screenDistance" : 60,
                    "gapFill" : True, 
                    "maxGapLength": 75, 
                    "noiseReduction" : False, 
                    "filterType" : "Median", 
                    "windowNoise" : 3,
                    "windowVelocity" : 20, 
                    "velocityThreshold" : 30, 
                    "mergeFixation" : True, 
                    "maxTimeBtwFixation" : 75,
                    "maxAngleBtwFixation" : 0.5, 
                    "discardShortFixation" : True,
                    "minDurationFixation" : 60
    }

    # POST pariticipant gaze data to the (R) eventDetection server
    url = 'http://localhost:6789/eventDetection'
    #url = 'http://127.0.0.1:6789/eventDetection'
    payload = {'config': config, 'data': ParticipantData.to_dict(orient="records")}
    response = requests.post(url, json = payload)
    
    # Create new a dataframe from the json response of the (R) eventDetection server
    participantDataWithEvents = pd.DataFrame.from_records(response.json())
    
    #Assign participantDataWithEvents to dataWithEvents or concate
    if dataWithEvents is None: 
        dataWithEvents = participantDataWithEvents
    else:
        dataWithEvents = pd.concat([dataWithEvents, participantDataWithEvents], axis=0)
        
    print('... Complete')

Applying fixation filter on the data for participant: P1
... Complete
Applying fixation filter on the data for participant: P2
... Complete
Applying fixation filter on the data for participant: P3
... Complete


In [11]:
# Preview gaze data with event information
dataWithEvents.head()

,Row,Respondent,Age,Gender,Group,Calibration,Timestamp,SourceStimuliName,EventSource.1,ET_TimeSignal,ET_CameraLeftX,ET_CameraLeftY,ET_CameraRightX,ET_CameraRightY,ET_ValidityLeft,ET_ValidityRight,GazeX,GazeY,SWA_Stimulus_Journalist_List_1,SWA_Stimulus_Status_invited1,SWA_Stimulus_Cancel_add,SWA_Stimulus_Add_1,SWA_Stimulus_Status_Request2,SWA_Stimulus_Navigation_2,SWA_Stimulus_Status_Published3,SWA_Stimulus_EventInfo_1,SWA_Stimulus_Confirm_add,SWA_Stimulus_Status_Published2,SWA_Stimulus_Journalist_List_6,SWA_Stimulus_Status_Published_6,SWA_Stimulus_Status_Published_5,SWA_Stimulus_Status Request1,SWA_Stimulus_EventInfo_2,SWA_Stimulus_Status_Published1,SWA_Stimulus_Journalist_List5,SWA_Stimulus_Question_Task1,SWA_Stimulus_Journalist_List4,SWA_Stimulus_EventInfo_3,SWA_Stimulus_Status_Published_4,SWA_Stimulus_Navigation_1,SWA_Stimulus_Journalist_selection,SWA_Stimulus_Event_Details,SWA_Stimulus_Journalist_List_3,SWA_Stimulus_Available_Journalists,SWA_Stimulus_Add_2,SWA_Stimulus_Journalist_List_2,SWA_Stimulus2_Current_Selection,SWA_Stimulus2_View button,SWA_Stimulus2_Navigation_2,SWA_Stimulus2_EventInfo,SWA_Stimulus2_Edit_button,SWA_Stimulus2_Submission_information,SWA_Stimulus2_Question_Task2,SWA_Stimulus2_Question,SWA_Stimulus2_Navigation_3,SWA_Stimulus2_Help_button,SWA_Stimulus2_Navigation_1,SWA_Stimulus2_Submission,SWA_Stimulus2_Question_Editor,SWA_Stimulus2_Published_Report_1,SWA_Stimulus2_Question_Control,InterpolatedGazeX,InterpolatedGazeY,InterpolatedDistance,ET_PupilLeft,ET_PupilRight,GazeVelocityAngle,FixID,Fixation Index by Stimulus,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,GazeAccelerationAngle,SacID,Saccade Index by Stimulus,Saccade Start,Saccade End,Saccade Duration,Saccade Amplitude,Saccade Peak Velocity,Saccade Peak Acceleration,Saccade Peak Deceleration,Saccade Direction
0,2,P1,0,OTHER,Default,Good,116.3702,SWA_Stimulus,1,124.985,0.6199,0.4231,0.4583,0.4169,0,0,949.0,470.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,949.0,470.0,631.0624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,P1,0,OTHER,Default,Good,124.6983,SWA_Stimulus,1,133.313,0.6199,0.4231,0.4582,0.4167,0,0,950.0,467.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,950.0,467.5,631.1380,4.4665,4.1949,24.9085,1.0,,949.1912,470.4823,120.5343,887.1217,766.5874,0.2832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,P1,0,OTHER,Default,Good,133.0560,SWA_Stimulus,1,141.671,0.6198,0.4231,0.4582,0.4166,0,0,957.0,455.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,957.0,455.5,631.1278,NaN,NaN,9.7702,1.0,,949.1912,470.4823,120.5343,887.1217,766.5874,0.2832,-1811.3025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,P1,0,OTHER,Default,Good,141.4015,SWA_Stimulus,1,150.015,0.6199,0.4229,0.4582,0.4166,0,0,956.0,470.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,956.0,470.0,631.1928,NaN,NaN,22.8842,1.0,,949.1912,470.4823,120.5343,887.1217,766.5874,0.2832,1571.3820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,P1,0,OTHER,Default,Good,149.7296,SWA_Stimulus,1,158.344,0.6198,0.4229,0.4581,0.4165,0,0,954.5,470.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,954.5,470.5,631.2959,4.4721,4.0730,8.3603,1.0,,949.1912,470.4823,120.5343,887.1217,766.5874,0.2832,-1743.9597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# export dataWithEvents as csv
dataWithEvents.to_csv("data/gazeDataWithAOIsAndEvents.csv",  index=False)